<a href="https://colab.research.google.com/github/psygrammer/about_python_dl_from_scrach_2/blob/main/notebooks/ch03/ch03_word2vec_sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. word2vec (solution)

* 싸이그래머 / 어바웃파이썬 : 딥러닝 NLP [1]
* 김무성

# 차례 
* 3.1 추론 기반 기법과 신경망
  - 3.1.1 통계 기반 기법의 문제점
  - 3.1.2 추론 기반 기법 개요
  - 3.1.3 신경망에서의 단어 처리
* 3.2 단순한 word2vec
  - 3.2.1 CBOW 모델의 추론 처리
  - 3.2.2 CBOW 모델의 학습
  - 3.2.3 word2vec의 가중치와 분산 표현
* 3.3 학습 데이터 준비
  - 3.3.1 맥락과 타깃
  - 3.3.2 원핫 표현으로 변환
* 3.4 CBOW 모델 구현
  - 3.4.1 학습 코드 구현
* 3.5 word2vec 보충
  - 3.5.1 CBOW 모델과 확률
  - 3.5.2 skip-gram 모델
  - 3.5.3 통계 기반 vs. 추론 기반
* 3.6 정리

------------------

In [1]:
!git clone https://github.com/psygrammer/about_python_dl_from_scrach_2.git

Cloning into 'about_python_dl_from_scrach_2'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 104 (delta 44), reused 33 (delta 5), pack-reused 0
Receiving objects: 100% (104/104), 746.54 KiB | 4.50 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [2]:
ls

about_python_dl_from_scrach_2/  sample_data/


In [ ]:
cd /content/about_python_dl_from_scrach_2/notebooks/ch03

In [ ]:
ls

------------------------

# 3.1 추론 기반 기법과 신경망
* 3.1.1 통계 기반 기법의 문제점
* 3.1.2 추론 기반 기법 개요
* 3.1.3 신경망에서의 단어 처리

## 3.1.1 통계 기반 기법의 문제점


## 3.1.2 추론 기반 기법 개요


## 3.1.3 신경망에서의 단어 처리

In [ ]:
import numpy as np

In [ ]:
# 입력
c = np.array([[1,0,0,0,0,0,0]])
c

In [ ]:
# 가중치
# 랜덤 초기화
W = np.random.randn(7,3)
W

-------------

# 3.2 단순한 word2vec
* 3.2.1 CBOW 모델의 추론 처리
* 3.2.2 CBOW 모델의 학습
* 3.2.3 word2vec의 가중치와 분산 표현


## 3.2.1 CBOW 모델의 추론 처리


## 3.2.2 CBOW 모델의 학습


## 3.2.3 word2vec의 가중치와 분산 표현

--------------

# 3.3 학습 데이터 준비
* 3.3.1 맥락과 타깃
* 3.3.2 원핫 표현으로 변환


## 3.3.1 맥락과 타깃


## 3.3.2 원핫 표현으로 변환

--------------

# 3.4 CBOW 모델 구현
* 3.4.1 학습 코드 구현


## 3.4.1 학습 코드 구현

-----------------

# 3.5 word2vec 보충
* 3.5.1 CBOW 모델과 확률
* 3.5.2 skip-gram 모델
* 3.5.3 통계 기반 vs. 추론 기반


## 3.5.1 CBOW 모델과 확률


## 3.5.2 skip-gram 모델


## 3.5.3 통계 기반 vs. 추론 기반

-----------------

# 3.6 정리

------------------
# 참고문헌
* [1] 밑바닥부터 시작하는 딥러닝 2
  - https://www.hanbit.co.kr/store/books/look.php?p_code=B8950212853
  - github - https://github.com/WegraLee/deep-learning-from-scratch-2